# QIIME2: Terminal Run (Common Final Process)
---
Terminal (Shell command line code) for QIIME2

This pipeline consists of 2 files (2 Processes)
1. [Main Process](Terminal_Run_Main_Process.ipynb)
2. [Database Process](Terminal_Run_DB_Process.ipynb)
3. [Common (Final) Process](Terminal_Run_Common_Process.ipynb)

**To reduce running time the [all processes](#) should be run parallelly**

Documentation of QIIME2 - https://docs.qiime2.org/2019.7/tutorials/overview
Official website - https://qiime2.org

## Step of 16S
<img src="step.png"/>

# 0. Set Variable
- The variable should be set in [all processes](#)
- To reduce running time the [all processes](#) should be run parallelly

In [ ]:
%env OTU_MANIFEST="otu.manifest.txt"
%env IN_DB_PERC=0.99

%env CPU_CORE=32
%env CPU_THREAD=0
%env CPU_JOB=-1

# 1 - 5. The Main Process
Run them in [Main Process](Terminal_Run_Main_Process.ipynb)

---
# 6 - 7, 10. The Database Process
Run them in [Database Process notebook](Terminal_Run_DB_Process.ipynb)

---

# 8. Clustering 

In [ ]:
%%bash
qiime vsearch cluster-features-closed-reference \
--i-table 05.1.table.qza \
--i-sequences 05.2.rep-seqs.qza \
--i-reference-sequences 06.3.ref-seqs.qza \
--p-perc-identity $IN_DB_PERC \
--p-threads $CPU_THREAD \
--verbose \
--o-clustered-table 07.1.table-cr-97.qza \
--o-clustered-sequences 07.2.rep-seqs-cr-97.qza \
--o-unmatched-sequences 07.3.unmatched-cr-97.qza

# 9. Chimeric Check
- https://docs.qiime2.org/2019.4/tutorials/chimera/

In [ ]:
%%bash
qiime vsearch uchime-denovo \
--i-table 07.1.table-cr-97.qza \
--i-sequences 07.2.rep-seqs-cr-97.qza \
--output-dir 08.uchime-dn-out

In [ ]:
%%bash

qiime feature-table filter-features \
--i-table 07.1.table-cr-97.qza \
--m-metadata-file 08.uchime-dn-out/nonchimeras.qza \
--o-filtered-table 08.1.table-nonchimeric-wo-borderline.qza

In [ ]:
%%bash

qiime feature-table filter-seqs \
--i-data 07.2.rep-seqs-cr-97.qza \
--m-metadata-file 08.uchime-dn-out/nonchimeras.qza \
--o-filtered-data 08.2.rep-seqs-nonchimeric-wo-borderline.qza

# 11. Feature Classify

In [ ]:
%%bash
qiime feature-classifier classify-sklearn \
--p-n-jobs $CPU_JOB \
--i-classifier 09.classifier.qza \
--i-reads 08.2.rep-seqs-nonchimeric-wo-borderline.qza \
--verbose \
--o-classification 10.2.taxonomy.qza

---
# Visualization
## 8. Clustering


In [ ]:
%%bash
qiime feature-table summarize \
--i-table 07.1.table-cr-97.qza \
--o-visualization vs07.1.table \
--m-sample-metadata-file $OTU_MANIFEST

In [ ]:
%%bash
qiime feature-table tabulate-seqs --i-data 07.2.rep-seqs-cr-97.qza --o-visualization vs07.2.rep-seqs
qiime feature-table tabulate-seqs --i-data 07.3.unmatched-cr-97.qza --o-visualization vs07.3.unmatched

## 9.1. Uchime filtering

In [ ]:
%%bash
qiime metadata tabulate \
--m-input-file 08.uchime-dn-out/stats.qza \
--o-visualization vs08.3.stats_uchime-denovo

qiime feature-table summarize \
--i-table 08.1.table-nonchimeric-wo-borderline.qza \
--o-visualization vs08.1.table-nonchimeric-wo-borderline

qiime feature-table tabulate-seqs \
--i-data 08.2.rep-seqs-nonchimeric-wo-borderline.qza \
--o-visualization vs08.2.rep-seqs

## 9.2. Visualization of alpha diversity 

In [ ]:
%%bash
qiime diversity alpha-rarefaction \
--i-table 08.1.table-nonchimeric-wo-borderline.qza \
--p-max-depth 32744 \
--p-metrics simpson \
--p-metrics shannon \
--p-metrics observed_otus \
--p-steps 1000 \
--o-visualization vs11.5.alpha-rarefaction.qzv \
--m-sample-metadata-file $OTU_MANIFEST

## 11. Predict taxonomy

In [ ]:
%%bash
qiime metadata tabulate \
--m-input-file 10.2.taxonomy.qza \
--o-visualization vs10.2.taxonomy.qzv

## 12. Create alpha diversity bar plot

In [ ]:
%%bash
qiime taxa barplot \
--i-table 08.1.table-nonchimeric-wo-borderline.qza \
--i-taxonomy 10.2.taxonomy.qza \
--o-visualization vs12.2.taxa-bar-plots.qzv \
--m-metadata-file $OTU_MANIFEST

## 13. Rarefy normalization

In [ ]:
%%bash
qiime feature-table rarefy \
--i-table 08.1.table-nonchimeric-wo-borderline.qza \
--p-sampling-depth 10000 \
--o-rarefied-table 14.rarefied_table.qza

## 14. PCoA

In [ ]:
%%bash
qiime diversity beta \
--i-table 14.rarefied_table.qza \
--p-metric braycurtis \
--p-n-jobs $CPU_JOB \
--o-distance-matrix 15.distance-matrix_braycurtis.qza

qiime diversity pcoa \
--i-distance-matrix 15.distance-matrix_braycurtis.qza \
--o-pcoa 16.1.pcoa_matrix_braycurtis_3D.qzv

qiime emperor plot \
--i-pcoa 16.1.pcoa_matrix_braycurtis_3D.qzv.qza \
--o-visualization vs17.pcoa_braycurtis.qzv \
--m-metadata-file $OTU_MANIFEST

## 15.  Show Sample-Taxonomy Table

In [ ]:
%%bash
qiime taxa barplot \
--i-table 08.1.table-nonchimeric-wo-borderline.qza \
--i-taxonomy 10.2.taxonomy.qza \
--m-metadata-file $OTU_MANIFEST
--o-visualization vs15.taxa-bar-plots_otuid.qzv